In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-for-oceanography/sample2.csv
/kaggle/input/ml-for-oceanography/test1.csv
/kaggle/input/ml-for-oceanography/train.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/ml-for-oceanography/train.csv')
test_data=pd.read_csv('/kaggle/input/ml-for-oceanography/test1.csv')
train_data.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,0.39
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,0.65
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,0.90
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,0.91
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,0.88


creating some features

In [3]:
# train_data['WPOW']= 0.5 * 1025 * 9.81 * train_data['WVHT(m)'] * train_data['WVHT(m)'] * train_data['DPD(sec)']

In [4]:
train_data.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,0.39
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,0.65
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,0.90
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,0.91
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,0.88


In [5]:
df=train_data.copy()

calculating the sine and cosine: Instead of using the raw wind direction feature, it may be helpful to convert it into its sine and cosine components to capture any directional patterns in the wind that affect wave height.

In [6]:
# Calculate the sine and cosine of wind direction
df['WDIR_sin'] = np.sin(np.radians(df['WDIR(degT)']))
df['WDIR_cos'] = np.cos(np.radians(df['WDIR(degT)']))
#wind speed cubed
df['WSCB']=np.power(df['WSPD(m/s)'],3)
#dominant wind power squared
df['DPD2']=np.power(df['DPD(sec)'],2)
#temperature differene between air and water
df['TDIF']=df['ATMP(degC)']-df['WTMP(degC)']
test_data['WDIR_sin'] = np.sin(np.radians(test_data['WDIR(degT)']))
test_data['WDIR_cos'] = np.cos(np.radians(test_data['WDIR(degT)']))
test_data['WSCB']=np.power(test_data['WSPD(m/s)'],3)
test_data['DPD2']=np.power(test_data['DPD(sec)'],2)
test_data['TDIF']=test_data['ATMP(degC)']-test_data['WTMP(degC)']

In [7]:
df.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),...,PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m),WDIR_sin,WDIR_cos,WSCB,DPD2,TDIF
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,...,1019.1,4.7,5.1,4.2,0.39,3.907311e-01,0.920505,27.000,17.3889,-0.4
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,...,1018.8,4.6,4.8,4.1,0.65,5.000000e-01,0.866025,24.389,27.6676,-0.2
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,...,1018.3,4.6,4.7,4.2,0.90,-2.449294e-16,1.000000,10.648,34.5744,-0.1
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,...,1018.1,4.6,4.6,4.2,0.91,-8.715574e-02,0.996195,15.625,34.5744,0.0
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,...,1018.3,4.5,4.6,4.0,0.88,-8.715574e-02,0.996195,42.875,39.0625,-0.1


In [8]:
df.shape

(3886, 22)

In [9]:
df.columns

Index(['ID', '#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR(degT)', 'WSPD(m/s)',
       'GST(m/s)', 'DPD(sec)', 'APD(sec)', 'MWD(degT)', 'PRES(hPa)',
       'ATMP(degC)', 'WTMP(degC)', 'DEWP(degC)', 'WVHT(m)', 'WDIR_sin',
       'WDIR_cos', 'WSCB', 'DPD2', 'TDIF'],
      dtype='object')

In [10]:
y=df['WVHT(m)']
df=df.drop(columns=['WVHT(m)'])
X = df.iloc[:, [2, 4, *range(6, df.shape[1])]]

In [11]:
X_test=test_data.iloc[:, [2, 4, *range(6, df.shape[1])]]

using ridge regression along with gridsearchCV


In [12]:
# Defining and training the ridge regressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
param_grid = {
    'alpha': [1.0, 10.0,12.5,15.0,17.5,20.0,30.0],
    'fit_intercept': [True, False]
}
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X, y)
print("Best hyperparameters:", grid_search.best_params_)
print("Test set mean squared error:", -grid_search.best_score_)

Best hyperparameters: {'alpha': 15.0, 'fit_intercept': False}
Test set mean squared error: 0.01657062752712713


making predictions

In [13]:
pred=grid_search.predict(X_test)

In [14]:
output=pd.DataFrame({'Id':test_data.ID,'WVHT(m)':pred})
output.to_csv('submission.csv',index=False)
print("Your submission was successful")

Your submission was successful


In [15]:
output.head()

,Id,WVHT(m)
0,2019722240,0.253014
1,2019722340,0.071036
2,201973140,0.174326
3,2019761040,0.126662
4,2019761140,0.185696
